In [2]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import os
import cv2
from os.path import splitext,basename
from keras.models import model_from_json
import scipy.fftpack # For FFT2 
# from google.colab.patches import cv2_imshow

In [3]:
def cv2_imshow(img):
    cv2.imshow("img",img)
    cv2.waitKey(0)
    cv2.destroyAllWindows()
def load_model(path):
    try:
        path = splitext(path)[0]
        with open('%s.json' % path, 'r') as json_file:
            model_json = json_file.read()
        model = model_from_json(model_json, custom_objects={})
        model.load_weights('%s.h5' % path)
        print("Loading model successfully...")
        return model
    except Exception as e:
        print(e)

In [4]:
wpod_net_path = r'C:\Users\Yatharth\Downloads\wpod-net.json'
wpod_net = load_model(wpod_net_path)

Loading model successfully...


In [5]:
class Label:
    def __init__(self, cl=-1, tl=np.array([0., 0.]), br=np.array([0., 0.]), prob=None):
        self.__tl = tl
        self.__br = br
        self.__cl = cl
        self.__prob = prob

    def __str__(self):
        return 'Class: %d, top left(x: %f, y: %f), bottom right(x: %f, y: %f)' % (
        self.__cl, self.__tl[0], self.__tl[1], self.__br[0], self.__br[1])

    def copy(self):
        return Label(self.__cl, self.__tl, self.__br)

    def wh(self): return self.__br - self.__tl

    def cc(self): return self.__tl + self.wh() / 2

    def tl(self): return self.__tl

    def br(self): return self.__br

    def tr(self): return np.array([self.__br[0], self.__tl[1]])

    def bl(self): return np.array([self.__tl[0], self.__br[1]])

    def cl(self): return self.__cl

    def area(self): return np.prod(self.wh())

    def prob(self): return self.__prob

    def set_class(self, cl):
        self.__cl = cl

    def set_tl(self, tl):
        self.__tl = tl

    def set_br(self, br):
        self.__br = br

    def set_wh(self, wh):
        cc = self.cc()
        self.__tl = cc - .5 * wh
        self.__br = cc + .5 * wh

    def set_prob(self, prob):
        self.__prob = prob

class DLabel(Label):
    def __init__(self, cl, pts, prob):
        self.pts = pts
        tl = np.amin(pts, axis=1)
        br = np.amax(pts, axis=1)
        Label.__init__(self, cl, tl, br, prob)

def getWH(shape):
    return np.array(shape[1::-1]).astype(float)

def IOU(tl1, br1, tl2, br2):
    wh1, wh2 = br1-tl1, br2-tl2
    assert((wh1 >= 0).all() and (wh2 >= 0).all())
    
    intersection_wh = np.maximum(np.minimum(br1, br2) - np.maximum(tl1, tl2), 0)
    intersection_area = np.prod(intersection_wh)
    area1, area2 = (np.prod(wh1), np.prod(wh2))
    union_area = area1 + area2 - intersection_area
    return intersection_area/union_area

def IOU_labels(l1, l2):
    return IOU(l1.tl(), l1.br(), l2.tl(), l2.br())

def nms(Labels, iou_threshold=0.5):
    SelectedLabels = []
    Labels.sort(key=lambda l: l.prob(), reverse=True)
    
    for label in Labels:
        non_overlap = True
        for sel_label in SelectedLabels:
            if IOU_labels(label, sel_label) > iou_threshold:
                non_overlap = False
                break

        if non_overlap:
            SelectedLabels.append(label)
    return SelectedLabels



def find_T_matrix(pts, t_pts):
    A = np.zeros((8, 9))
    for i in range(0, 4):
        xi = pts[:, i]
        xil = t_pts[:, i]
        xi = xi.T
        
        A[i*2, 3:6] = -xil[2]*xi
        A[i*2, 6:] = xil[1]*xi
        A[i*2+1, :3] = xil[2]*xi
        A[i*2+1, 6:] = -xil[0]*xi

    [U, S, V] = np.linalg.svd(A)
    H = V[-1, :].reshape((3, 3))
    return H

def getRectPts(tlx, tly, brx, bry):
    return np.matrix([[tlx, brx, brx, tlx], [tly, tly, bry, bry], [1, 1, 1, 1]], dtype=float)

def normal(pts, side, mn, MN):
    pts_MN_center_mn = pts * side
    pts_MN = pts_MN_center_mn + mn.reshape((2, 1))
    pts_prop = pts_MN / MN.reshape((2, 1))
    return pts_prop

# Reconstruction function from predict value into plate crpoped from image
def reconstruct(I, Iresized, Yr, lp_threshold):
    # 4 max-pooling layers, stride = 2
    net_stride = 2**4
    side = ((208 + 40)/2)/net_stride

    # one line and two lines license plate size
    one_line = (470, 110)
    two_lines = (280, 200)

    Probs = Yr[..., 0]
    Affines = Yr[..., 2:]

    xx, yy = np.where(Probs > lp_threshold)
    # CNN input image size 
    WH = getWH(Iresized.shape)
    # output feature map size
    MN = WH/net_stride

    vxx = vyy = 0.5 #alpha
    base = lambda vx, vy: np.matrix([[-vx, -vy, 1], [vx, -vy, 1], [vx, vy, 1], [-vx, vy, 1]]).T
    labels = []
    labels_frontal = []

    for i in range(len(xx)):
        x, y = xx[i], yy[i]
        affine = Affines[x, y]
        prob = Probs[x, y]

        mn = np.array([float(y) + 0.5, float(x) + 0.5])

        # affine transformation matrix
        A = np.reshape(affine, (2, 3))
        A[0, 0] = max(A[0, 0], 0)
        A[1, 1] = max(A[1, 1], 0)
        # identity transformation
        B = np.zeros((2, 3))
        B[0, 0] = max(A[0, 0], 0)
        B[1, 1] = max(A[1, 1], 0)

        pts = np.array(A*base(vxx, vyy))
        pts_frontal = np.array(B*base(vxx, vyy))

        pts_prop = normal(pts, side, mn, MN)
        frontal = normal(pts_frontal, side, mn, MN)

        labels.append(DLabel(0, pts_prop, prob))
        labels_frontal.append(DLabel(0, frontal, prob))
        
    final_labels = nms(labels, 0.1)
    final_labels_frontal = nms(labels_frontal, 0.1)

#     print(final_labels_frontal)
    if(final_labels_frontal==[]):
        print("No License plate is founded!")
        return None
    assert final_labels_frontal, "No License plate is founded!"

    # LP size and type
    out_size, lp_type = (two_lines, 2) if ((final_labels_frontal[0].wh()[0] / final_labels_frontal[0].wh()[1]) < 1.7) else (one_line, 1)

    TLp = []
    Cor = []
    if len(final_labels):
        final_labels.sort(key=lambda x: x.prob(), reverse=True)
        for _, label in enumerate(final_labels):
            t_ptsh = getRectPts(0, 0, out_size[0], out_size[1])
            ptsh = np.concatenate((label.pts * getWH(I.shape).reshape((2, 1)), np.ones((1, 4))))
            H = find_T_matrix(ptsh, t_ptsh)
            Ilp = cv2.warpPerspective(I, H, out_size, borderValue=0)
            TLp.append(Ilp)
            Cor.append(ptsh)
    return final_labels, TLp, lp_type, Cor

def detect_lp(model, I, max_dim, lp_threshold):
    min_dim_img = min(I.shape[:2])
    factor = float(max_dim) / min_dim_img
    w, h = (np.array(I.shape[1::-1], dtype=float) * factor).astype(int).tolist()
    Iresized = cv2.resize(I, (w, h))
    T = Iresized.copy()
    T = T.reshape((1, T.shape[0], T.shape[1], T.shape[2]))
    Yr = model.predict(T)
    Yr = np.squeeze(Yr)
    #print(Yr.shape)
    try:
        L, TLp, lp_type, Cor = reconstruct(I, Iresized, Yr, lp_threshold)
    except:
        return None
    return L, TLp, lp_type, Cor


In [6]:
def imclearborder(imgBW, radius):

    # Given a black and white image, first find all of its contours
    imgBWcopy = imgBW.copy()
    contours,hierarchy = cv2.findContours(imgBWcopy.copy(), cv2.RETR_LIST, 
        cv2.CHAIN_APPROX_SIMPLE)

    # Get dimensions of image
    imgRows = imgBW.shape[0]
    imgCols = imgBW.shape[1]    

    contourList = [] # ID list of contours that touch the border

    # For each contour...
    for idx in np.arange(len(contours)):
        # Get the i'th contour
        cnt = contours[idx]

        # Look at each point in the contour
        for pt in cnt:
            rowCnt = pt[0][1]
            colCnt = pt[0][0]

            # If this is within the radius of the border
            # this contour goes bye bye!
            check1 = (rowCnt >= 0 and rowCnt < radius) or (rowCnt >= imgRows-1-radius and rowCnt < imgRows)
            check2 = (colCnt >= 0 and colCnt < radius) or (colCnt >= imgCols-1-radius and colCnt < imgCols)

            if check1 or check2:
                contourList.append(idx)
                break

    for idx in contourList:
        cv2.drawContours(imgBWcopy, contours, idx, (0,0,0), -1)

    return imgBWcopy


#### bwareaopen definition
def bwareaopen(imgBW, areaPixels):
    # Given a black and white image, first find all of its contours
    imgBWcopy = imgBW.copy()
    contours,hierarchy = cv2.findContours(imgBWcopy.copy(), cv2.RETR_LIST, 
        cv2.CHAIN_APPROX_SIMPLE)

    # For each contour, determine its total occupying area
    for idx in np.arange(len(contours)):
        # area = cv2.contourArea(contours[idx])
        x, y, w, h = cv2.boundingRect(contours[idx])
        area = w*h
        if (area >= 0 and area <= areaPixels):
            cv2.drawContours(imgBWcopy, contours, idx, (0,0,0), -1)

    return imgBWcopy


def auto_canny(image, sigma=0.33):
    # compute the median of the single channel pixel intensities
    v = np.median(image)
 
    # apply automatic Canny edge detection using the computed median
    lower = int(max(0, (1.0 - sigma) * v))
    upper = int(min(255, (1.0 + sigma) * v))
    edged = cv2.Canny(image, lower, upper)
 
    # return the edged image
    return edged


def ClearThresold(wrap):
    img = wrap.copy()
    # img = cv2.resize(img,(6*img.shape[1],6*img.shape[0]))

    # Number of rows and columns
    rows = img.shape[0]
    cols = img.shape[1]

    # Remove some columns from the beginning and end
    # img = img[:, 59:cols-20]

    # Number of rows and columns
    rows = img.shape[0]
    cols = img.shape[1]

    # Convert image to 0 to 1, then do log(1 + I)
    imgLog = np.log1p(np.array(img, dtype="float") / 255)

    # Create Gaussian mask of sigma = 10
    M = 2*rows + 1
    N = 2*cols + 1
    sigma = 10
    (X,Y) = np.meshgrid(np.linspace(0,N-1,N), np.linspace(0,M-1,M))
    centerX = np.ceil(N/2)
    centerY = np.ceil(M/2)
    gaussianNumerator = (X - centerX)**2 + (Y - centerY)**2

    # Low pass and high pass filters
    Hlow = np.exp(-gaussianNumerator / (2*sigma*sigma))
    Hhigh = 1 - Hlow

    # Move origin of filters so that it's at the top left corner to
    # match with the input image
    HlowShift = scipy.fftpack.ifftshift(Hlow.copy())
    HhighShift = scipy.fftpack.ifftshift(Hhigh.copy())

    # Filter the image and crop
    If = scipy.fftpack.fft2(imgLog.copy(), (M,N))
    Ioutlow = scipy.real(scipy.fftpack.ifft2(If.copy() * HlowShift, (M,N)))
    Iouthigh = scipy.real(scipy.fftpack.ifft2(If.copy() * HhighShift, (M,N)))

    # Set scaling factors and add
    gamma1 = 0.3
    gamma2 = 1.5
    Iout = gamma1*Ioutlow[0:rows,0:cols] + gamma2*Iouthigh[0:rows,0:cols]

    # Anti-log then rescale to [0,1]
    Ihmf = np.expm1(Iout)
    Ihmf = (Ihmf - np.min(Ihmf)) / (np.max(Ihmf) - np.min(Ihmf))
    Ihmf2 = np.array(255*Ihmf, dtype="uint8")
    # cv2_imshow(Ihmf2)

    # Threshold the image - Anything below intensity 80 gets set to white
    Ithresh = Ihmf2 < 80
    Ithresh = 255*Ithresh.astype("uint8")
    Ithresh = cv2.copyMakeBorder(Ithresh, 10, 10, 10, 10, cv2.BORDER_CONSTANT, None, (0,0,0))


    # Clear off the border.  Choose a border radius of 5 pixels
    Iclear = imclearborder(Ithresh, 5)

    # Eliminate regions that have areas below 120 pixels
    Iopen = bwareaopen(Iclear, 0.006*(Iclear.shape[0]*Iclear.shape[1]))

    # Show all images
    # cv2_imshow(img)
    # cv2_imshow(Ihmf2)
    # cv2_imshow(Ithresh)
    # cv2_imshow(Iclear)
    # cv2_imshow(Iopen)
    return Iopen

def Get_Segmented(Iopen):
    ctrs, _ = cv2.findContours(Iopen.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    sorted_ctrs = sorted(ctrs, key=lambda ctr: cv2.boundingRect(ctr)[0])
    img_area = Iopen.shape[0]*Iopen.shape[1]
    # print(Iopen.shape[0],Iopen.shape[1])
    letters = []
    for i, ctr in enumerate(sorted_ctrs):
        area = cv2.contourArea(ctr)
        x, y, w, h = cv2.boundingRect(ctr)
        mx= max(w,h)
        roi_area = w*h
        roi_ratio = roi_area/img_area
        if mx*mx >= Iopen.shape[0]*Iopen.shape[0]*0.03 and w>0.0135 * Iopen.shape[1] and h>0.075 * Iopen.shape[0] and h <0.75 * Iopen.shape[0] and w <0.3 * Iopen.shape[1] :
        # cv2.rectangle(Iopen,(x,y),( x + w, y + h ),(90,0,255),1)
        # print(w,h)
            letters.append([x,y,w,h])
        # if ((roi_ratio >= 0.015) and (roi_ratio < 0.09))  and area > 0.0001*img_area :
        #   if ((h>0.2*w) and (4*w>=h)):
        #     cv2.rectangle(Iopen,(x,y),( x + w, y + h ),(90,0,255),1)
        #     letters.append([x,y,w,h])
#     cv2_imshow(Iopen)
    # print("***")
    letters = refine_contours(letters)
    letters = sorted(letters, key=lambda ctr: ctr[0])
    segmented = []
    for coord in letters:
        [x, y, w, h] = coord
        # print("x, y, w, h = ",x, y, w, h)
        # print(image_result.shape[0],image_result.shape[1])
        seg_letter = Iopen[(int)(max(0, y - 0.2 * h)) : (int)(min(Iopen.shape[0], y + 1.2 * h)), (int)(max(0, x - 0.05 * w)) : (int)(min(Iopen.shape[1], x + 1.05 * w))]

        # giving some black border to the images
        limit = max(5,(int)((h-w+5)/2))
        seg_letter = cv2.copyMakeBorder(seg_letter, 5, 5, limit, limit, cv2.BORDER_CONSTANT)
        segmented.append(seg_letter)
    return segmented
# Check if a rectangle present inside other
# helps is selecting the outermost rectangle
def rectContains(rect, pt):
    ret = rect[0] <= pt[0] <= rect[0] + rect[2] and rect[1] <= pt[1] <= rect[1] + rect[3]
    return ret

# Find all the biggest rectangles/contours in the image
def refine_contours(list_coord):
    ret_cnt = list_coord.copy()
    
    for i in list_coord:
        for j in list_coord:
            if rectContains(i, [j[0], j[1]]) and rectContains(i, [j[0] + j[2], j[1]]) and rectContains(i, [j[0], j[1] + j[3]]) and rectContains(i, [j[0] + j[2], j[1] + j[3]]) and i != j:
                if j in ret_cnt:
                    ret_cnt.remove(j)
                    
    return ret_cnt


In [7]:
def get_plate(image, Dmax=608, Dmin=228):
    vehicle = preprocess_image(image)
    ratio = float(max(vehicle.shape[:2])) / min(vehicle.shape[:2])
    side = int(ratio * Dmin)
    bound_dim = min(side, Dmax)
    try:
        _ , LpImg, _, cor = detect_lp(wpod_net, vehicle, bound_dim, lp_threshold=0.5)
    except:
        return None,None
    return LpImg, cor


def preprocess_image(image,resize=False):
    img = image
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = img / 255
    if resize:
        img = cv2.resize(img, (224,224))
    return img

In [8]:
# utility functions

import cv2
import numpy as np
from matplotlib import pyplot as plt
import scipy.fftpack # For FFT2 


def cv2_imshow(img):
	cv2.imshow("img",img)
	cv2.waitKey(0)
	cv2.destroyAllWindows()



def predict_from_model(image,model):
	image = cv2.resize(image,(80,80))
	image = np.stack((image,)*3, axis=-1)
	prediction = np.argmax(model.predict(image[np.newaxis,:]))
	return prediction
def four_point_transform(image, pts):

	rect = order_points(pts,image.shape)
	(tl, tr, br, bl) = rect

	widthA = np.sqrt(((br[0] - bl[0]) ** 2) + ((br[1] - bl[1]) ** 2))
	widthB = np.sqrt(((tr[0] - tl[0]) ** 2) + ((tr[1] - tl[1]) ** 2))
	maxWidth = max(int(widthA), int(widthB))

	heightA = np.sqrt(((tr[0] - br[0]) ** 2) + ((tr[1] - br[1]) ** 2))
	heightB = np.sqrt(((tl[0] - bl[0]) ** 2) + ((tl[1] - bl[1]) ** 2))
	maxHeight = max(int(heightA), int(heightB))

	dst = np.array([[0, 0], [maxWidth - 1, 0], [maxWidth - 1, maxHeight - 1], [0, maxHeight - 1]], dtype = "float32")

	M = cv2.getPerspectiveTransform(rect, dst)
	warped = cv2.warpPerspective(image, M, (maxWidth, maxHeight))

	return warped
def order_points(pts,shape):
	rect = np.zeros((4, 2), dtype = "float32")

	s = pts.sum(axis = 1)
	rect[0] = pts[np.argmin(s)]
	rect[0] = [max(rect[0][0]-2,0),max(rect[0][1]-2,0)]
	rect[2] = pts[np.argmax(s)]
	rect[2] = [min(rect[2][0]+2,shape[1]-1),min(rect[2][1]+2,shape[0]-1)]

	diff = np.diff(pts, axis = 1)
	rect[1] = pts[np.argmin(diff)]
	rect[1] = [min(rect[1][0]+2,shape[1]-1),max(rect[1][1]-2,0)]

	rect[3] = pts[np.argmax(diff)]
	rect[3] = [max(rect[3][0]-2,0),min(rect[3][1]+2,shape[0]-1)]

# 	print(rect)
	return rect
# Now let's create a mask for this image
def createMask(size, hull):
	(rows, cols)  = size
	# black image
	mask = np.zeros((rows, cols), dtype=np.uint8)
	# blit our contours onto it in white color
	cv2.drawContours(mask, hull, 0, 255, -1)
	return mask
# Check if a rectangle present inside other
# helps is selecting the outermost rectangle
def rectContains(rect, pt):
	ret = rect[0] <= pt[0] <= rect[0] + rect[2] and rect[1] <= pt[1] <= rect[1] + rect[3]
	return ret

# Find all the biggest rectangles/contours in the image
def refine_contours(list_coord):
	ret_cnt = list_coord.copy()

	for i in list_coord:
		for j in list_coord:
			if rectContains(i, [j[0], j[1]]) and rectContains(i, [j[0] + j[2], j[1]]) and rectContains(i, [j[0], j[1] + j[3]]) and rectContains(i, [j[0] + j[2], j[1] + j[3]]) and i != j:
				if j in ret_cnt:
					ret_cnt.remove(j)
		
	return ret_cnt


def CreateHull(plate):
	gray_plate = cv2.cvtColor( plate , cv2.COLOR_BGR2GRAY )
	gray_plate = cv2.fastNlMeansDenoising(gray_plate , h= 3,templateWindowSize = 7,searchWindowSize = 21)
	# cv2_imshow(gray_plate)

	gray_plate = cv2.GaussianBlur(gray_plate, (3, 3), 0)

	ret3,thresh = cv2.threshold(gray_plate,0,255,cv2.THRESH_BINARY+cv2.THRESH_OTSU)
	kernel = np.ones((7,7),np.uint8)
	thresh = cv2.erode(thresh,kernel,iterations = 1)
	# cv2_imshow(thresh)#imp

	contours,_ = cv2.findContours(thresh, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
	final_cnt = None
	max_area = 0
	for cnt in contours:
		area = cv2.contourArea(cnt)
		if(area>max_area):
			max_area = area
			final_cnt = cnt
	drawing = np.zeros((thresh.shape[0], thresh.shape[1], 3), np.uint8)
	hull = []
	hull.append(cv2.convexHull(final_cnt, False))
	hull2 = cv2.convexHull(final_cnt, False)
	color = (255, 0, 0)
	cv2.drawContours(drawing, hull,0, color, 1, 8)
	# cv2_imshow(drawing)
	return hull

  

#### imclearborder definition

def imclearborder(imgBW, radius):

	# Given a black and white image, first find all of its contours
	imgBWcopy = imgBW.copy()
	contours,hierarchy = cv2.findContours(imgBWcopy.copy(), cv2.RETR_LIST, 
		cv2.CHAIN_APPROX_SIMPLE)

	# Get dimensions of image
	imgRows = imgBW.shape[0]
	imgCols = imgBW.shape[1]	

	contourList = [] # ID list of contours that touch the border

	# For each contour...
	for idx in np.arange(len(contours)):
        		# Get the i'th contour
        		cnt = contours[idx]

        # Look at each point in the contour
	for pt in cnt:
		rowCnt = pt[0][1]
		colCnt = pt[0][0]

		# If this is within the radius of the border
		# this contour goes bye bye!
		check1 = (rowCnt >= 0 and rowCnt < radius) or (rowCnt >= imgRows-1-radius and rowCnt < imgRows)
		check2 = (colCnt >= 0 and colCnt < radius) or (colCnt >= imgCols-1-radius and colCnt < imgCols)

		if check1 or check2:
			contourList.append(idx)
			break

	for idx in contourList:
		cv2.drawContours(imgBWcopy, contours, idx, (0,0,0), -1)

	return imgBWcopy


#### bwareaopen definition
def bwareaopen(imgBW, areaPixels):
	# Given a black and white image, first find all of its contours
	imgBWcopy = imgBW.copy()
	contours,hierarchy = cv2.findContours(imgBWcopy.copy(), cv2.RETR_LIST, 
		cv2.CHAIN_APPROX_SIMPLE)

	# For each contour, determine its total occupying area
	for idx in np.arange(len(contours)):
		# area = cv2.contourArea(contours[idx])
		x, y, w, h = cv2.boundingRect(contours[idx])
		area = w*h
		if (area >= 0 and area <= areaPixels):
	  		cv2.drawContours(imgBWcopy, contours, idx, (0,0,0), -1)

	return imgBWcopy


def auto_canny(image, sigma=0.33):
	# compute the median of the single channel pixel intensities
	v = np.median(image)
 
	# apply automatic Canny edge detection using the computed median
	lower = int(max(0, (1.0 - sigma) * v))
	upper = int(min(255, (1.0 + sigma) * v))
	edged = cv2.Canny(image, lower, upper)
 
	# return the edged image
	return edged


def ClearThresold(wrap):
	img = wrap.copy()
	# img = cv2.resize(img,(6*img.shape[1],6*img.shape[0]))

	# Number of rows and columns
	rows = img.shape[0]
	cols = img.shape[1]

	# Remove some columns from the beginning and end
	# img = img[:, 59:cols-20]

	# Number of rows and columns
	rows = img.shape[0]
	cols = img.shape[1]

	# Convert image to 0 to 1, then do log(1 + I)
	imgLog = np.log1p(np.array(img, dtype="float") / 255)

	# Create Gaussian mask of sigma = 10
	M = 2*rows + 1
	N = 2*cols + 1
	sigma = 10
	(X,Y) = np.meshgrid(np.linspace(0,N-1,N), np.linspace(0,M-1,M))
	centerX = np.ceil(N/2)
	centerY = np.ceil(M/2)
	gaussianNumerator = (X - centerX)**2 + (Y - centerY)**2

	# Low pass and high pass filters
	Hlow = np.exp(-gaussianNumerator / (2*sigma*sigma))
	Hhigh = 1 - Hlow

	# Move origin of filters so that it's at the top left corner to
	# match with the input image
	HlowShift = scipy.fftpack.ifftshift(Hlow.copy())
	HhighShift = scipy.fftpack.ifftshift(Hhigh.copy())

	# Filter the image and crop
	If = scipy.fftpack.fft2(imgLog.copy(), (M,N))
	Ioutlow = scipy.real(scipy.fftpack.ifft2(If.copy() * HlowShift, (M,N)))
	Iouthigh = scipy.real(scipy.fftpack.ifft2(If.copy() * HhighShift, (M,N)))

	# Set scaling factors and add
	gamma1 = 0.3
	gamma2 = 1.5
	Iout = gamma1*Ioutlow[0:rows,0:cols] + gamma2*Iouthigh[0:rows,0:cols]

	# Anti-log then rescale to [0,1]
	Ihmf = np.expm1(Iout)
	Ihmf = (Ihmf - np.min(Ihmf)) / (np.max(Ihmf) - np.min(Ihmf))
	Ihmf2 = np.array(255*Ihmf, dtype="uint8")
# 	cv2_imshow(Ihmf2)
# 	plt.hist(Ihmf2.ravel(),256,[0,256]); plt.show()
	pix = [] 
	for i in range(256):
		pix.append(0)
	for i in range(Ihmf2.shape[0]):
		for j in range(Ihmf2.shape[1]):
			pix[Ihmf2[i][j]]+=1
	# print(pix)
	mx = max(pix)
	# print(max(pix))
# 	print(pix.index(mx))
	limit = pix.index(mx)

	# Threshold the image - Anything below intensity 80 gets set to white
	Ithresh = Ihmf2 < limit*0.8
	Ithresh = 255*Ithresh.astype("uint8")
	Ithresh = cv2.copyMakeBorder(Ithresh, 10, 10, 10, 10, cv2.BORDER_CONSTANT, None, (0,0,0))


	# Clear off the border.	Choose a border radius of 5 pixels
	Iclear = imclearborder(Ithresh, 5)

	# Eliminate regions that have areas below 120 pixels
	Iopen = bwareaopen(Iclear, 0.006*(Iclear.shape[0]*Iclear.shape[1]))

	# Show all images
	# cv2_imshow(img)
	# cv2_imshow(Ihmf2)
	# cv2_imshow(Ithresh)
	# cv2_imshow(Iclear)
	# cv2_imshow(Iopen)
	return Iopen


def refine_answer(ans_lis):
	if ans_lis[1] == 'H' and (ans_lis[0]=='W' or ans_lis[0] == 'H' and ans_lis[0] == 'N'):
		ans_lis[0] = 'M'
	
	if ans_lis[1] == 'B' and (ans_lis[0]=='M' or ans_lis[0] == 'N'):
		ans_lis[0] = 'W'

	for i in range(len(ans_lis)):
		if ans_lis[i] == 'O':
			ans_lis[i] = '0'

	check = ['0', '1', '2', '3', '4', '5', '6', '7', '8', '9']
	if len(ans_lis) == 10:
		if ans_lis[len(ans_lis) - 1] not in check:
			if ans_lis[len(ans_lis) - 1] == 'B':
				ans_lis[len(ans_lis) - 1] = '8'

			if ans_lis[len(ans_lis) - 1] == 'I':
				ans_lis[len(ans_lis) - 1] = '1'

		if ans_lis[len(ans_lis) - 2] not in check:
			if ans_lis[len(ans_lis) - 2] == 'B':
				ans_lis[len(ans_lis) - 2] = '8'

			if ans_lis[len(ans_lis) - 1] == 'I':
				ans_lis[len(ans_lis) - 1] = '1'

		if ans_lis[len(ans_lis) - 3] not in check:
			if ans_lis[len(ans_lis) - 3] == 'B':
				ans_lis[len(ans_lis) - 3] = '8'

			if ans_lis[len(ans_lis) - 1] == 'I':
				ans_lis[len(ans_lis) - 1] = '1'

		if ans_lis[len(ans_lis) - 4] not in check:
			if ans_lis[len(ans_lis) - 4] == 'B':
				ans_lis[len(ans_lis) - 4] = '8'

			if ans_lis[len(ans_lis) - 1] == 'I':
				ans_lis[len(ans_lis) - 1] = '1'
	return ans_lis


def Get_Segmented(Iopen):
	ctrs, _ = cv2.findContours(Iopen.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
	sorted_ctrs = sorted(ctrs, key=lambda ctr: cv2.boundingRect(ctr)[0])
	img_area = Iopen.shape[0]*Iopen.shape[1]
	# print(Iopen.shape[0],Iopen.shape[1])
	letters = []
	for i, ctr in enumerate(sorted_ctrs):
		area = cv2.contourArea(ctr)
		x, y, w, h = cv2.boundingRect(ctr)
		mx= max(w,h)
		roi_area = w*h
		roi_ratio = roi_area/img_area
		if mx*mx >= Iopen.shape[0]*Iopen.shape[0]*0.03 and w>0.0135 * Iopen.shape[1] and h>0.07 * Iopen.shape[0] and h <0.70 * Iopen.shape[0] and  w <0.25 * Iopen.shape[1]:
			# cv2.rectangle(Iopen,(x,y),( x + w, y + h ),(90,0,255),1)
			# print(w,h)
			letters.append([x,y,w,h])
			# if ((roi_ratio >= 0.015) and (roi_ratio < 0.09))  and area > 0.0001*img_area :
			#   if ((h>0.2*w) and (4*w>=h)):
			#     cv2.rectangle(Iopen,(x,y),( x + w, y + h ),(90,0,255),1)
			#     letters.append([x,y,w,h])
# 	cv2_imshow(Iopen)
	# print("***")
	letters = refine_contours(letters)
	letters = sorted(letters, key=lambda ctr: ctr[0])
	segmented = []
	for coord in letters:
		[x, y, w, h] = coord
		# print("x, y, w, h = ",x, y, w, h)
		# print(image_result.shape[0],image_result.shape[1])
		seg_letter = Iopen[(int)(max(0, y - 0.2 * h)) : (int)(min(Iopen.shape[0], y + 1.2 * h)), (int)(max(0, x - 0.05 * w)) : (int)(min(Iopen.shape[1], x + 1.05 * w))]

		# giving some black border to the images
		limit = max(5,(int)((h-w+5)/2))
		seg_letter = cv2.copyMakeBorder(seg_letter, 5, 5, limit, limit, cv2.BORDER_CONSTANT)
		segmented.append(seg_letter)
	return segmented



In [9]:

json_file = open('MobileNets_character_recognition.json', 'r')
loaded_model_json = json_file.read()
json_file.close()
model = model_from_json(loaded_model_json)
model.load_weights("License_character_recognition_weight.h5")    
characters = ["0",   "1",   "2",   "3",   "4",   "5",   "6",   "7",   "8",   "9",   "A",   "B",   "C",   "D",   "E",   "F",   "G",   "H",   "I",   "J",   "K",   "L",   "M",   "N",   "O",   "P",   "Q",   "R",   "S",   "T",   "U",   "V",   "W",   "X",   "Y",   "Z"]  


In [17]:
def get_text(float_plates):
    plates = []
    answer = []
    wpod = False
    scale_factor = 3
#     float_plates, coord = get_plate(image)
    if(float_plates is not None):
#         print("WPOD***************************")
        wpod = True
        scale_factor = 3
        for img in float_plates:
            temp = (img*255)
            temp=np.uint8(temp) 
#             cv2_imshow(temp)
            plates.append(temp)  
        # return answer
    
    for plate in plates:
        # path = "Chevrolet-Captiva-524767e.jpg_0348_0049_0207_0136_0054.png"
        # plate = cv2.imread(path)
        plate = cv2.resize(plate,(scale_factor*plate.shape[1],scale_factor*plate.shape[0]))
        # cv2_imshow(plate)

        hull = CreateHull(plate)

        mask = createMask(plate.shape[0:2], hull)
        # cv2_imshow(mask)

        contours,_ = cv2.findContours(mask, cv2.RETR_TREE, cv2.CHAIN_APPROX_SIMPLE)
        # print((contours))
        # cv2.drawContours(plate, contours, -1, (0,0,255), 1)
        # cv2_imshow(plate)

        temp = contours[0]
        hull2 = cv2.convexHull(temp, False)
        perimeter = cv2.arcLength(hull2,True) #it gives the perimeter of each shape(true is for closed shape)
        approxCnt = cv2.approxPolyDP(hull2,0.05*perimeter,True) #this will give coridinates of all the corner points
        No_of_points = len(approxCnt)
        if No_of_points == 4 and not wpod:
            wrap = four_point_transform(plate, np.array([approxCnt[0][0],approxCnt[1][0],approxCnt[2][0],approxCnt[3][0]]))

        else:
            wrap = plate
        wrap = cv2.fastNlMeansDenoisingColored(wrap,None,10,10,7,21)
        wrap = cv2.cvtColor(wrap,cv2.COLOR_BGR2GRAY)
#         cv2_imshow(wrap) 
        Iopen = ClearThresold(wrap)
        # cv2_imshow(Iopen)
        segmented = Get_Segmented(Iopen)
#         for seg in segmented:
#             cv2_imshow(seg)
#         cv2_imshow(plate)



        ANSWER = []
#         z= 1
        for image in segmented:
#             cv2.imshow("image"+str(z),image)
#             z+=1
            ANSWER.append(characters[predict_from_model(image,model)])
#         cv2.waitKey(0)
#         cv2.destroyAllWindows()
#         ANSWER = refine_answer(ANSWER)
        print(ANSWER)
        answer.append(ANSWER)
    return answer

In [18]:
cap = cv2.VideoCapture(r'C:\Users\Yatharth\Desktop\MOSAIC PS2\ps2\test_video\video.mp4')
# cap = cv2.VideoCapture(r'C:\Users\Yatharth\Downloads\Top 10 best 4x4s and SUVs to buy.mp4')

lap = 0       
wrap, coord = None,None
while (cap.isOpened()):
    ret, img = cap.read()
    if ret == True:
        if cv2.waitKey(25) & 0xFF == ord('q'):
            break
        if(lap%20==0):
            wrap, coord = get_plate(img)
        if(wrap is not None):
            for i in range(len(wrap)):
                cv2.rectangle(img,((int)(coord[i][0][0]),(int)(coord[i][1][0])),( (int)(coord[i][0][2]),(int)(coord[i][1][2])),(90,0,255),3)
        cv2.imshow('original video', img)
    else:
        break
    lap += 1

cap.release()
cv2.destroyAllWindows()

No License plate is founded!
No License plate is founded!
No License plate is founded!
No License plate is founded!
No License plate is founded!
No License plate is founded!
No License plate is founded!
No License plate is founded!
No License plate is founded!
No License plate is founded!
No License plate is founded!
No License plate is founded!
No License plate is founded!
No License plate is founded!
No License plate is founded!
No License plate is founded!
No License plate is founded!
No License plate is founded!
No License plate is founded!
No License plate is founded!
No License plate is founded!
No License plate is founded!
No License plate is founded!
No License plate is founded!
No License plate is founded!
No License plate is founded!
No License plate is founded!
No License plate is founded!
No License plate is founded!
No License plate is founded!
No License plate is founded!
No License plate is founded!
No License plate is founded!
No License plate is founded!
No License pla